In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets
import torchvision.models as models
import torch.nn as nn
import torch.optim as optim
import glob
import os
import numpy as np

In [2]:
#from torch.utils.tensorboard import SummaryWriter
from tensorboardX import SummaryWriter
writer=SummaryWriter('/home/are/data_cervical/pytorch_running/third/resnet50/Adam/run2')

In [3]:
transform = transforms.Compose(
    [transforms.Resize((224,224)),transforms.ToTensor(),
     transforms.Normalize((0.5941, 0.3818, 0.3370), (0.1508 ,0.1404, 0.1222))])
train_dataset=torchvision.datasets.ImageFolder('/home/are/data_cervical/pytorch_running/3_data/caijian2/train/',transform)
valid_dataset=torchvision.datasets.ImageFolder('/home/are/data_cervical/pytorch_running/3_data/caijian2/valid/',transform)
train_dataset.class_to_idx

{'ONE': 0, 'THREE': 1, 'TWO': 2}

In [4]:
train_loader=torch.utils.data.DataLoader(train_dataset,batch_size=32,shuffle=True,num_workers=5)
valid_loader=torch.utils.data.DataLoader(valid_dataset,batch_size=32,shuffle=True,num_workers=5)

In [10]:
save_path='/home/are/data_cervical/pytorch_running/third/resnet50/Adam/epoch_160.pth'
load=torch.load(save_path)
model_weight=load['net']
epoch_start=load['epoch']
optimizer_weight=load['optimizer']

In [14]:
model_ft=models.resnet50()
num_ftrs=model_ft.fc.in_features
model_ft.fc=nn.Linear(num_ftrs,3) 
model_ft.load_state_dict(model_weight)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model_ft=model_ft.to(device)
print(model_ft)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=F

In [15]:
criterion=nn.CrossEntropyLoss()
optimizer=torch.optim.Adam(model_ft.parameters(),weight_decay=1e-4,lr=0.001,betas=(0.9,0.99))
optimizer.load_state_dict(optimizer_weight) #之前把weight也留下来了

In [16]:
save_path='/home/are/data_cervical/pytorch_running/third/resnet50/Adam'
import time
dataloader={'train':train_loader,'valid':valid_loader}
dataset={'train':train_dataset,'valid':valid_dataset}
best_model_wts=model_ft.state_dict()
best_acc=0.0
num_epochs=200
for epoch in range(161,num_epochs):
    since=time.time()
    print('epoch:',epoch)
    print('-'*10)
        
    for phase in ['train','valid']:
            
        if phase=='train':
            model_ft.train(True)
        else:
            model_ft.train(False)
        running_loss=0.0
        running_corrects=0
        
        for i, data in enumerate(dataloader[phase], 0):
                
            inputs, labels = data
            inputs=torch.autograd.Variable(inputs.to(device))  #数据和标签要放到GPU上
            labels=torch.autograd.Variable(labels.to(device))
                
            optimizer.zero_grad()
                # forward + backward + optimize
            outputs = model_ft(inputs)
            loss = criterion(outputs, labels)
            _,preds=torch.max(outputs.data,1)
            if phase=='train':
                loss.backward()
                optimizer.step()
            running_loss+=loss.item()
            running_corrects+=(torch.sum(preds==labels.data)).item()
        print(running_loss)
        print(running_corrects)
       
           
        print(len(dataset[phase]))
        epoch_loss=running_loss/len(dataset[phase])
        epoch_acc=running_corrects/len(dataset[phase])
        if phase=='train':
            
            writer.add_scalars('loss_train_val',{'train':epoch_loss},epoch)
            writer.add_scalars('acc_train_val',{'train':epoch_acc},epoch)
            if epoch%20==0: #每隔一段时间保存一次
                state={'net':model_ft.state_dict(),'optimizer':optimizer.state_dict(),'epoch':epoch}
                dir=os.path.join(save_path,'epoch_{}.pth'.format(epoch))
                torch.save(state,dir)
        else:
            
            writer.add_scalars('loss_train_val',{'valid':epoch_loss},epoch)
            writer.add_scalars('acc_train_val',{'valid':epoch_acc},epoch)
        print('{} loss: {:.8f} acc:{:.8f}'.format(phase,epoch_loss,epoch_acc))
        if phase=='valid' and epoch_acc>best_acc:
            best_acc=epoch_acc
            best_model_wts=model_ft.state_dict()
            
    time_elapsed= time.time()-since
   
    print('training time:',time_elapsed)
model_ft.load_state_dict(best_model_wts)
torch.save(best_model_wts,os.path.join(save_path,'resnet50_weight.pth'))
torch.save(model_ft,os.path.join(save_path,'resnet50.pth'))
print('best acc',best_acc)

epoch: 161
----------
21.72862965799868
17523
17780
train loss: 0.00122208 acc:0.98554556
6.623684790916741
4370
4444
valid loss: 0.00149048 acc:0.98334833
training time: 86.45144891738892
epoch: 162
----------
20.635797191411257
17556
17780
train loss: 0.00116062 acc:0.98740157
8.387592878192663
4350
4444
valid loss: 0.00188740 acc:0.97884788
training time: 88.23655986785889
epoch: 163
----------
20.033713191747665
17558
17780
train loss: 0.00112676 acc:0.98751406
5.8424581084400415
4383
4444
valid loss: 0.00131468 acc:0.98627363
training time: 90.75522875785828
epoch: 164
----------
21.319866406731308
17547
17780
train loss: 0.00119909 acc:0.98689539
4.933122305199504
4395
4444
valid loss: 0.00111006 acc:0.98897390
training time: 90.20448660850525
epoch: 165
----------
19.545930860564113
17573
17780
train loss: 0.00109932 acc:0.98835771
8.226676881313324
4361
4444
valid loss: 0.00185119 acc:0.98132313
training time: 90.19661664962769
epoch: 166
----------
23.08040574984625
17544
1778

In [25]:
#test
model=models.resnet50()
num_ftrs=model.fc.in_features
model.fc=nn.Linear(num_ftrs,3) 
model.load_state_dict(torch.load(os.path.join(save_path,'resnet50_weight.pth')))
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model=model.to(device)
running_corrects=0
c00=0
c01=0
c02=0
c10=0
c11=0
c12=0
c20=0
c21=0
c22=0
for i, data in enumerate(dataloader['valid'], 0):
    
    inputs, labels = data
    
    inputs=torch.autograd.Variable(inputs.to(device))  #数据和标签要放到GPU上
    labels=torch.autograd.Variable(labels.to(device))                                                                                        
    outputs = model(inputs)
    _,preds=torch.max(outputs.data,1)
    for i in range(len(labels.data)):
        if labels.data[i]==0:
            if preds[i]==0:
                c00+=1
            if preds[i]==1:
                c01+=1
            if preds[i]==2:
                c02+=1
        if labels.data[i]==1:
            if preds[i]==0:
                c10+=1
            if preds[i]==1:
                c11+=1
            if preds[i]==2:
                c12+=1
        if labels.data[i]==2:
            if preds[i]==0:
                c20+=1
            if preds[i]==1:
                c21+=1
            if preds[i]==2:
                c22+=1
    
    running_corrects+=(torch.sum(preds==labels.data)).item()
    
epoch_acc=running_corrects/len(dataset['valid'])
print(c00,c01,c02)
print(c10,c11,c12)
print(c20,c21,c22)
print(running_corrects)
print(epoch_acc)
print(len(dataset['valid']))

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=F

863 3 28
3 894 24
8 15 2606
4363
0.9817731773177317
4444
